#### Data Loader to download daily yahoo price data daily and persist in csv files

In [14]:
import pickle
import random_forest
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import datetime
from datetime import date
import yfinance as yf

pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_csv("clusters.csv")
risk_df = df[df.cluster==0]

In [19]:
def get_current_price(symbol):
    yf.pdr_override()
    data = pdr.get_data_yahoo(symbol, start=date.today() - datetime.timedelta(1), end=date.today())
    return data['Close'].values

def get_historical(symbol):
    end_date = date.today() - datetime.timedelta(2)
    start_date = end_date - datetime.timedelta(365)
    yf.pdr_override()
    data = pdr.get_data_yahoo(symbol, start=start_date, end=end_date)
    return data

# Calculate proportion change from one column to another
def p_change(df, col1, col2):
    col1_s = pd.Series(df[col1])
    col2_s = pd.Series(df[col2])
    pc_c1_c2 = []
    for idx, value in enumerate(col1_s):
        pc = col2_s[idx] - col1_s[idx]
        pc = pc / col1_s[idx]
        pc_c1_c2.append(pc)
    return pc_c1_c2

#### Download historical price data from yahoo

In [142]:
# save historical data so don't have to reload it
symbols = ['BTC','ETH','ADA','DOGE','XRP','HEX','BCH','LTC','LINK','MATIC','THETA']
symbols = symbols + ['XLM','VET','ETC','TRX','FIL','XMR','EOS','ALGO','CRO','TFUEL','BSV']
symbols = symbols + ['NEO','XTZ','MIOTA','LUNA1','MKR','ATOM1','KSM','BTT1','HBAR','RUNE','CHZ']
symbols = symbols + ['WAVES','DCR','CEL','ZEC','DASH','HOT1','XEM','QNT','ZIL']
symbols = symbols + ['ENJ','BAT','STX1','MANA','SNX','XWC','ZEN','BTG','NANO','BNT','DGB','ONE2']
symbols = symbols + ['QTUM','ARRR','ONT','SC','ZRX','OMG','ANKR','ICX','RVN','BCD','XVG']
symbols = symbols + ['CKB','IOST','RSR','MAID','KNC','HNC','LRC']
symbols = symbols + ['LSK','KAVA','VTHO','RLC','GNO','BAND','STORJ']
symbols = symbols + ['MCO','ABBC','FUN','OXT','WAXP','SNT','FET','IOTX','NKN','ANT','REP','BTS','CVC','DERO']
symbols = symbols + ['MLN','TOMO','AVA','ARDR','XHV','ETN','BCN']

import yfinance as yf

end_date = date.today() - datetime.timedelta(1)
start_date = end_date - datetime.timedelta(365)

for i, symbol in enumerate(symbols):
    print(symbol,'start',start_date,'end',end_date)
    data = yf.download(symbol + '-USD', start=start_date, end=end_date)
    data.to_csv('price-data/' + symbol + '.csv')

BTC start 2020-08-03 end 2021-08-03
[*********************100%***********************]  1 of 1 completed
ETH start 2020-08-03 end 2021-08-03
[*********************100%***********************]  1 of 1 completed
ADA start 2020-08-03 end 2021-08-03
[*********************100%***********************]  1 of 1 completed
DOGE start 2020-08-03 end 2021-08-03
[*********************100%***********************]  1 of 1 completed
XRP start 2020-08-03 end 2021-08-03
[*********************100%***********************]  1 of 1 completed
HEX start 2020-08-03 end 2021-08-03
[*********************100%***********************]  1 of 1 completed
BCH start 2020-08-03 end 2021-08-03
[*********************100%***********************]  1 of 1 completed
LTC start 2020-08-03 end 2021-08-03
[*********************100%***********************]  1 of 1 completed
LINK start 2020-08-03 end 2021-08-03
[*********************100%***********************]  1 of 1 completed
MATIC start 2020-08-03 end 2021-08-03
[**************

#### augment historical prices with differentials for classification

In [143]:
for i, symbol in enumerate(symbols):
    try:
        df = pd.read_csv('price-data/' + symbol + '.csv')
        df.drop(df.tail(1).index,inplace=True) # drop last row

        # prepare data for classifier
        sma15c = df.Close.rolling(15,1).mean()
        sma30c = df.Close.rolling(30,1).mean()
        pc_sma15c_sma30c = []
        for idx, value in enumerate(sma15c):
            pc = sma30c[idx] - sma15c[idx]
            pc = pc / sma30c[idx]
            pc_sma15c_sma30c.append(pc)
        df['pc_sma15c_sma30c'] = pc_sma15c_sma30c
        df['pc_open_close'] = p_change(df, 'Open', 'Close')
        df['pc_high_low'] = p_change(df, 'High', 'Low')
        df['pc_low_close'] = p_change(df, 'Low', 'Close')
        df['symbol'] = symbol
        column_order = ['symbol','Date','High','Low','Open','Close','Volume','Adj Close','pc_sma15c_sma30c','pc_open_close','pc_high_low','pc_low_close']
        df = df[column_order]
        
        # save data for classifier
        df.to_csv('price-data/' + symbol + '.csv')
    except:
        print(symbol, "error loading")

In [69]:
# get current prices for symbols in cluster

prices = []

for index, row in risk_df.iterrows():
    symbol = row['coin'] + '-USD'
    
    # get current price
    try:
        current_price = get_current_price(symbol)
        prices.append(current_price[0])
    except:
        prices.append(0)   
    
risk_df['price'] = prices

,Unnamed: 0,coin,return,variance,cluster,price
28,28,BTT1,6.718235,108.406759,0,0
36,37,HOT1,5.883760,74.008961,0,0
40,41,ENJ,8.187812,124.015507,0,0
43,44,MANA,5.866293,71.159058,0,0
57,59,ANKR,7.549854,129.732200,0,0
82,84,FET,5.404705,84.151166,0,0
92,94,AVA,6.046762,68.050562,0,0


In [73]:
# run classifier
import pickle
import random_forest

classifier_filename = 'classification.sav'
regressor_filename = 'SVR.sav'

# load models
classifier = pickle.load(open(classifier_filename, 'rb'))
regressor = pickle.load(open(regressor_filename, 'rb'))

C:\Users\steve\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVR from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [75]:
risk_df.columns

Index(['Unnamed: 0', 'coin', 'return', 'variance', 'cluster', 'price'], dtype='object')

In [168]:
symbol = 'BTC'
# load data for classifier
features = ['symbol','Date','Close','pc_sma15c_sma30c', 'pc_open_close', 'pc_high_low', 'pc_low_close']
df = pd.read_csv('price-data/' + symbol + '.csv')[features]

# get last row for prediction
x = df.iloc[-1]

classifier.predict(x)

False

In [258]:
features = ['symbol','Date','Close','pc_sma15c_sma30c', 'pc_open_close', 'pc_high_low', 'pc_low_close']

# get trends from classifier
for index, row in risk_df.iterrows():
    symbol = row['coin']
    
    # load data for classifier
    df = pd.read_csv('price-data/' + symbol + '.csv')[features]
    
    # get last row for prediction
    x = df.iloc[-1]
    classifier.predict(x)    
    
#     # get current price
#     try:
#         current_price = classifier(symbol)
#         prices.append(current_price[0])
#     except:
#         prices.append(0)   

risk_df['price'] = prices

def get_predicted_price(df, model, days):

    X = np.array(df[["Close"]])
    X = X[:df.shape[0] - days]
    try:
        preds = model.predict(X)
        result = preds[len(preds)-1]
    except:
        result = 0
    return result
    
# get predicted prices for regression
predicted_prices = []
for index, row in risk_df.iterrows():
    symbol = row['coin']
    col_names = ['Date','Close']
    file_name = 'price-data/' + symbol + '.csv'
    df = pd.read_csv(file_name)[col_names]
    df = df.set_index(pd.DatetimeIndex(df["Date"].values))
    try:
        days = [1,7,365]
        preds = []
        for d in days:
            pred_price = get_predicted_price(df, regressor, d)
            preds.append(pred_price)
        predicted_prices.append(preds)
    except:
        predicted_prices.append([0,0,0])

pred_prices_df = pd.DataFrame(predicted_prices, columns=['oneday','oneweek','oneyear'])
pred_prices_df['coin'] = risk_df['coin'].values
final_df = pd.merge(risk_df,pred_prices_df)
final_df

,Unnamed: 0,coin,return,variance,cluster,price,oneday,oneweek,oneyear
0,28,BTT1,6.718235,108.406759,0,0,0.242172,0.242152,0.241993
1,37,HOT1,5.883760,74.008961,0,0,0.242418,0.242396,0.000000
2,41,ENJ,8.187812,124.015507,0,0,0.341761,0.335500,0.253669
3,44,MANA,5.866293,71.159058,0,0,0.291838,0.292066,0.247703
4,59,ANKR,7.549854,129.732200,0,0,0.249367,0.247232,0.242446
5,84,FET,5.404705,84.151166,0,0,0.270359,0.265063,0.248300
6,94,AVA,6.046762,68.050562,0,0,0.397775,0.382502,0.300374


In [249]:
risk_df['coin'].values

array(['BTT1', 'HOT1', 'ENJ', 'MANA', 'ANKR', 'FET', 'AVA'], dtype=object)

In [187]:
symbol = 'ABBC'
features = ['symbol','Date','Close','pc_sma15c_sma30c', 'pc_open_close', 'pc_high_low', 'pc_low_close']
df = pd.read_csv('price-data/' + symbol + '.csv')[features]
    
# get last price for regression
future_days = [1,7,365]
pred_price = []

for d in future_days:
    reg_df = df.copy()
    reg_df = reg_df.set_index(pd.DatetimeIndex(df["Date"].values))
    X = np.array(reg_df[["Close"]])
    X = X[:reg_df.shape[0] - d]    
    preds = regressor.predict(X)  
    pred_price.append(preds[len(preds) - 1])
    
    
pred_price

[0.25492025109656424, 0.25709757910613007, 0.2536038658081452]

In [260]:
from optimizer import *

In [283]:
# run portfolio optimization
# load porfolio data for optimization
df = pd.read_csv('optimizer-data/annualreturns.csv') 
R = pd.read_csv('optimizer-data/modelreturns.csv')

coin_list = risk_df['coin'].values
df = df[df.coin.isin(coin_list)]
R = R[R.coin.isin(coin_list)]

In [271]:
df.columns

Index(['coin', 'return'], dtype='object')

,return,variance
coin,,
BTC,1.179525,0.566068
ETC,1.572186,1.471802
ADA,2.057490,1.319614
DOGE,3.589713,5.659477
XRP,0.972929,1.502262
...,...,...
AVA,6.046762,68.050562
ARDR,1.064982,1.548637
XHV,2.467752,3.044253


In [266]:
df = df.join(risk_df, on=["coin"])
R = R.join(risk_df, on=["coin"])

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat